In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Colab Notebooks/Masterarbeit/two_WL_NTK

Mounted at /content/drive
/content/drive/MyDrive/Colab Notebooks/Masterarbeit/two_WL_NTK


In [2]:
import numpy as np
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
import pandas as pd
import numpy

def load_kernel_matrix_from_splits(base_path, dataset_size, size_splits, number_splits, file_name_start="NTK_"):

  l = lambda x : x*size_splits 
  h = lambda x : (1+x)*size_splits if (not x == number_splits) else dataset_size-1

  kernel_matrix = np.empty((dataset_size, dataset_size))

  for i in range(number_splits+1):
    for j in range(number_splits+1):    
      split_k = np.load(kernel_path + f"/NTK_{h(i)}_{h(j)}.npy")
      kernel_matrix[l(i):h(i),l(j):h(j)] = split_k
    
  return kernel_matrix


def kernel_SVM(kernel_matrix, ys):
  C_list = np.logspace(-2, 4, 120)
  svc = SVC(kernel='precomputed', max_iter=5e5)
  clf = GridSearchCV(svc, {'C' : C_list}, verbose=0, return_train_score=True)
  clf.fit(kernel_matrix, ys)

  return pd.DataFrame(clf.cv_results_)

In [3]:
# GCNN data
data_path = f"/content/drive/MyDrive/MasterarbeitData/Preprocessed/MUTAG"
kernel_path = f"/content/drive/MyDrive/MasterarbeitData/Kernels/MUTAG/GCNN"

kernel_matrix = load_kernel_matrix_from_splits(kernel_path, 188, 50, 3)
ys = np.load(data_path + f"/ys.npy")

kernel_matrix = kernel_matrix[:-1,:-1]
ys = ys[:-1]
ys = np.squeeze(ys, 1)

# estimate kernel svm
svc, result = kernel_SVM(kernel_matrix, ys)
result[result["mean_test_score"] == result["mean_test_score"].max()]

0
1
2
3


/usr/local/lib/python3.8/dist-packages/sklearn/svm/_base.py:284: ConvergenceWarning: Solver terminated early (max_iter=500000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/svm/_base.py:284: ConvergenceWarning: Solver terminated early (max_iter=500000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/svm/_base.py:284: ConvergenceWarning: Solver terminated early (max_iter=500000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/svm/_base.py:284: ConvergenceWarning: Solver terminated early (max_iter=500000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/svm/_base.py:284: ConvergenceWarning: Solver terminated early (max_iter=500000).  Con

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,...,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
115,0.008314,0.00143,0.000595,0.000095,6285.203269,{'C': 6285.203269240097},0.947368,0.815789,0.837838,0.864865,...,0.86074,0.046024,1,0.825503,0.838926,0.86,0.84,0.86,0.844886,0.013356


In [10]:
svc = SVC(kernel='precomputed', max_iter=5e5, C=6285.203269)
svc.fit(kernel_matrix, ys)
svc.predict(kernel_matrix)

array([1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0,
       1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0,
       0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0,
       1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1], dtype=int32)

In [4]:
# Two_wl Kernel
data_path = f"/content/drive/MyDrive/MasterarbeitData/Preprocessed/MUTAG"
kernel_path = f"/content/drive/MyDrive/MasterarbeitData/Kernels/MUTAG/TWO_WL"

kernel_matrix = load_kernel_matrix_from_splits(kernel_path, 188, 10, 18)
ys = np.load(data_path + f"/ys.npy")

kernel_matrix = kernel_matrix[:-1,:-1]
ys = ys[:-1]
ys = np.squeeze(ys, 1)

# estimate kernel svm
result = kernel_SVM(kernel_matrix, ys)
result[result["mean_test_score"] == result["mean_test_score"].max()]

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18


/usr/local/lib/python3.8/dist-packages/sklearn/svm/_base.py:284: ConvergenceWarning: Solver terminated early (max_iter=500000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/svm/_base.py:284: ConvergenceWarning: Solver terminated early (max_iter=500000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/svm/_base.py:284: ConvergenceWarning: Solver terminated early (max_iter=500000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/svm/_base.py:284: ConvergenceWarning: Solver terminated early (max_iter=500000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/svm/_base.py:284: ConvergenceWarning: Solver terminated early (max_iter=500000).  Con

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,...,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
2,0.019809,0.03753,0.000523,0.000376,0.012614,{'C': 0.012613637220374436},0.868421,0.815789,0.783784,0.837838,...,0.828734,0.028032,1,0.845638,0.818792,0.766667,0.753333,0.753333,0.787553,0.037757
